<a target="_blank" href="https://colab.research.google.com/drive/1CAg-Kb7QvhkuIgfyjxNodvFL9jzGR7oJ?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
! pip install pip==21.3.1
! pip install ultralytics==8.0.53
! pip install clearml==1.9.3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import clearml
from clearml import Task
from clearml import Model
from ultralytics import YOLO
import onnx
import onnxruntime as ort

In [2]:
clearml.browser_login()

<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀


In [3]:
# get best performing clearml task
task = Task.get_task(
    project_name="Pedestrian-Detection-YOLOv8", 
    task_name="yolov8s_imgsz1024_epochs50",
)
# download the best model from clearml server
best_model_id = task.output_models_id["best"]
clearml_model = Model(best_model_id)
checkpoint_fp = clearml_model.get_local_copy()
print(checkpoint_fp)

/root/.clearml/cache/storage_manager/global/83a4f9ffd734970ca8a30c38c37a8b18.best.pt


In [ ]:
%%shell
cp /content/drive/MyDrive/Reza/Projects/Pedestrian-Detection/data/citypersons.zip .
unzip -q citypersons.zip

In [4]:
# load yolo model from downloaded checkpoint
model = YOLO(checkpoint_fp)

In [5]:
# evaluate model performance on validation set
model.val("/content/citypersons/dataset.yaml")

Ultralytics YOLOv8.0.53 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/citypersons/valid/labels.cache... 441 images, 10 backgrounds, 0 corrupt: 100%|██████████| 451/451 [00:00<?, ?it/s]
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [08:24<00:00, 17.41s/it]
                   all        451       4164      0.721      0.516      0.593      0.405
                 rider        451       1007      0.674       0.38      0.441      0.289
            pedestrian        451       3157      0.768      0.653      0.744      0.521
Speed: 9.1ms preprocess, 1076.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val4


In [7]:
img_fp = "/content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png"
%timeit result = model.predict(img_fp)


image 1/1 /content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png: 512x1024 1 pedestrian, 692.2ms
Speed: 1.6ms preprocess, 692.2ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png: 512x1024 1 pedestrian, 685.1ms
Speed: 1.6ms preprocess, 685.1ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png: 512x1024 1 pedestrian, 669.4ms
Speed: 1.6ms preprocess, 669.4ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png: 512x1024 1 pedestrian, 674.1ms
Speed: 1.6ms preprocess, 674.1ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_009561_leftImg8bit.png: 512x1024 1 pedestrian, 666.4ms
Speed: 1.6ms pr

712 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# save the checkpoint to GDrive
! cp "{checkpoint_fp}" /content/drive/MyDrive/Reza/Projects/Pedestrian-Detection/models/yolov8s_imgsz1024_mAP50_0.593.pt